In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re
import gensim as gs

In [2]:
names = [405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430]

In [3]:
len(names)

48

In [4]:
sm = 0
for i in names:
    f = open('data/categories/{}_urls.txt'.format(i))
    lines = f.readlines()
    sm += len(lines)
    f.close()
print(sm)

7238


In [5]:
tree = np.load('data/tree.npy')

In [6]:
def dfs(tree, parents, childrens):
    mas = []
    if tree['children'] != None:
        for i in tree['children']:
            parents[i['a_attr']['title'].split('.')[0]] = tree['a_attr']['title'].split('.')[0]
            mas.extend([i['a_attr']['title'].split('.')[0]])
            dfs(i, parents, childrens)
    childrens[tree['a_attr']['title'].split('.')[0]] = mas

In [7]:
parents = {'0':'0'}
childrens = {}
for i in tree:
    parents[i['a_attr']['title'].split('.')[0]] = '0'
    dfs(i, parents, childrens)

In [8]:
def getData(tree, names, allUrls, parent_numb, st):
    dataUrls = []
    dataText = []
    dataCategories = []
    
    number = int(tree['a_attr']['title'].split('.')[0])
    if number in names and number in files:
        f = open('data/filterData/categories/{}.txt'.format(number))
        lines = f.readlines()
        categUrls = [i.split(', ', 1)[0] for i in lines]
        categText = [i.split(', ', 1)[1] for i in lines]
        f.close()
        
        for i in range(len(categUrls)):
            if categUrls[i] not in allUrls:
                dataUrls.extend([categUrls[i]])
                dataText.append([categText[i].replace('\n', '')])
                dataCategories.extend([number])

                allUrls.add(categUrls[i])
            
    if len(tree['children']) != 0:
        for i in tree['children']:
            a, b, c = getData(i, names, allUrls, number, st)
            dataUrls.extend(a)
            dataText.extend(b)
            dataCategories.extend(c)
    
    else:
        if number in names and number in files \
            and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
            dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()

    return dataUrls, dataText, dataCategories

In [9]:
files = glob.glob('data/filterData/categories/*')
files = [int(i[27:].split('.')[0]) for i in files]

In [10]:
allUrls = set()
dataUrls = []
dataText = []
dataCategories = []
st = '   '
parent_numb = 1
for i in tree:
    a, b, c = getData(i, names, allUrls, parent_numb, st)
    dataUrls.extend(a)
    dataText.extend(b)
    dataCategories.extend(c)

In [11]:
dataUrls = np.array(dataUrls)
dataText = np.array(dataText,  dtype='O')
dataCategories = np.array(dataCategories)

In [12]:
len(set(dataCategories))

47

In [13]:
len(dataUrls)

6255

In [14]:
dataAns = [(dataUrls[i], dataCategories[i]) for i in range(dataUrls.shape[0])]

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
trainText = np.load('data/filterData/patTrainText.npy')
trainAns = np.load('data/filterData/patTrainAns.npy')
testText = np.load('data/filterData/patTestText.npy')
testAns = np.load('data/filterData/patTestAns.npy')

In [17]:
trainCategory = [float(i[1]) for i in trainAns]
testCategory = [float(i[1]) for i in testAns]

In [18]:
trainTexts = trainText
testTexts = testText

### w2v

In [19]:
trainTokens = np.array([i.split() for i in trainTexts])
testTokens = np.array([i.split() for i in testTexts])

In [20]:
%%time
model_w2v = gs.models.Word2Vec(trainTokens)

CPU times: user 43.6 s, sys: 224 ms, total: 43.8 s
Wall time: 16.3 s


In [21]:
words = np.array(model_w2v.wv.index2word)

In [22]:
words_vec = np.array([model_w2v[i] for i in words])

In [23]:
from sklearn.cluster import KMeans

In [68]:
n_clusters = 4000

In [69]:
kmeans = KMeans(n_clusters=n_clusters, n_jobs=1, n_init=1)#, random_state=0)

In [70]:
kmeans.fit(words_vec)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4000, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [71]:
words_labels = kmeans.labels_

In [72]:
cluster_center = kmeans.cluster_centers_

In [73]:
words_dist = np.copy(words_vec)
for i in range(n_clusters):
    words_dist[words_labels == i] -= cluster_center[i]

In [74]:
words_dict = dict()
for i, word in enumerate(words):
    words_dict[word] = np.array([words_labels[i], words_dist[i]], dtype='O')

In [75]:
trainFeatures = np.zeros((trainTokens.shape[0]), dtype='O')
for i in range(trainTokens.shape[0]):
    trainFeatures[i] = ''
    for j in trainTokens[i]:
        if j in model_w2v:
            trainFeatures[i] += str(words_dict[j][0]) + ' '
    
testFeatures = np.zeros((testTokens.shape[0]), dtype='O')
for i in range(testTokens.shape[0]):
    testFeatures[i] = ''
    for j in testTokens[i]:
        if j in model_w2v:
            testFeatures[i] += str(words_dict[j][0]) + ' '

In [76]:
tfidfTransformer = TfidfVectorizer()
trainFeatures  = tfidfTransformer.fit_transform(trainFeatures)
testFeatures = tfidfTransformer.transform(testFeatures)

In [77]:
trainFeatures.shape

(5003, 3990)

# CLF

In [78]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  13.767823934555054
0.740207833733
0.8129496402877698


In [53]:
_time = time.time()
clf = LogisticRegression()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  6.138647079467773
0.734612310152
0.8041566746602717


In [55]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.09091615676879883
0.674660271783
0.7921662669864109


In [56]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.009717941284179688
0.565947242206
0.6442845723421263


In [57]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.007257223129272461
0.675459632294
0.7721822541966427


In [58]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  67.46084332466125
0.116706634692
0.1486810551558753


In [59]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  61.66218447685242
0.72342126299
0.8113509192645884
